In [1]:
# =====================================================
# FAKE NEWS DETECTION – MiniLM + PROMPT TUNING + LIAR 2
# Dataset: chengxuphd/liar2
# CONFIG: GIỐNG HỆT DISTILBERT ĐỂ SO SÁNH
# =====================================================

# 1. INSTALL
!pip install -q transformers datasets peft accelerate bitsandbytes scikit-learn pandas numpy psutil

# 2. IMPORT
import os, re, shutil, psutil, warnings
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, EarlyStoppingCallback,
    DataCollatorWithPadding
)
from peft import PromptTuningConfig, PromptTuningInit, get_peft_model, TaskType
from google.colab import drive

warnings.filterwarnings("ignore")

# 3. GPU INFO
device_name = "CPU"
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    print(f"Device: {device_name} | VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("Device: CPU")

# 4. MOUNT GOOGLE DRIVE
try:
    drive.mount('/content/drive', force_remount=True)
except ValueError:
    print("Drive có thể đã được mount, tiếp tục...")

# 5. OUTPUT DIR
OUTPUT_DIR = "/content/drive/MyDrive/LIAR2_MiniLM_PromptTuning"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 6. LOAD DATA (LIAR 2)
print("Loading 'chengxuphd/liar2' dataset...")
try:
    dataset = load_dataset("chengxuphd/liar2")
except Exception as e:
    print(f"Lỗi tải dataset: {e}. Đang thử load bản gốc...")
    dataset = load_dataset("liar")

# Chuyển sang DataFrame
train_df = pd.DataFrame(dataset["train"])
val_df   = pd.DataFrame(dataset["validation"])
test_df  = pd.DataFrame(dataset["test"])

# --- XỬ LÝ NHÃN (6 Lớp -> 2 Lớp) ---
if train_df["label"].max() > 1:
    print("Mapping 6 labels -> Binary (Fake/Real)...")
    # 0: false, 1: half-true, 2: mostly-true, 3: true, 4: barely-true, 5: pants-fire
    def map_labels(label):
        if label in [0, 4, 5]: return 1 # FAKE
        return 0 # REAL

    train_df["label"] = train_df["label"].apply(map_labels)
    val_df["label"]   = val_df["label"].apply(map_labels)
    test_df["label"]  = test_df["label"].apply(map_labels)

# --- TẠO NỘI DUNG ĐẦU VÀO ---
# Kiểm tra tên cột văn bản (text/statement)
text_col = "statement" if "statement" in train_df.columns else "text"
context_col = "subject" if "subject" in train_df.columns else "context"

def create_content(row):
    stmt = str(row.get(text_col, ""))
    spkr = str(row.get("speaker", ""))
    ctxt = str(row.get(context_col, ""))
    return f"Statement: {stmt} | Speaker: {spkr} | Context: {ctxt}"

train_df["content"] = train_df.apply(create_content, axis=1)
val_df["content"]   = val_df.apply(create_content, axis=1)
test_df["content"]  = test_df.apply(create_content, axis=1)

# Tạo Dataset
train_dataset = Dataset.from_pandas(train_df[["content", "label"]])
val_dataset   = Dataset.from_pandas(val_df[["content", "label"]])
test_dataset  = Dataset.from_pandas(test_df[["content", "label"]])

dataset_dict = DatasetDict({"train": train_dataset, "validation": val_dataset, "test": test_dataset})

# 7. TOKENIZER (MiniLM)
MODEL_NAME = "nreimers/MiniLM-L6-H384-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(batch["content"], truncation=True, max_length=384, padding=False)

tokenized = dataset_dict.map(tokenize_fn, batched=True, remove_columns=["content"])
tokenized = tokenized.rename_column("label", "labels")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 8. LOAD MODEL & CONFIG PROMPT TUNING
base_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# --- CẤU HÌNH FIX CỨNG CHO MINILM (L6-H384) ---
# MiniLM khác DistilBERT ở Hidden Size (384 vs 768)
n_layers = 6
hidden_size = 384  # <-- ĐIỂM KHÁC BIỆT DUY NHẤT VỀ KIẾN TRÚC
n_heads = 12

peft_config = PromptTuningConfig(
    task_type=TaskType.SEQ_CLS,
    prompt_tuning_init=PromptTuningInit.TEXT,
    prompt_tuning_init_text="Classify validity of the statement:",
    num_virtual_tokens=32,
    tokenizer_name_or_path=MODEL_NAME,
    num_layers=n_layers,
    token_dim=hidden_size, # Bắt buộc phải là 384 cho MiniLM
    num_attention_heads=n_heads
)

model = get_peft_model(base_model, peft_config)
print("\nTham số huấn luyện (Prompt Tuning):")
model.print_trainable_parameters()

# 9. CLASS WEIGHTS
labels_array = train_df["label"].values
class_weights = compute_class_weight("balanced", classes=np.array([0,1]), y=labels_array)
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# 10. METRICS
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=0)
    acc = accuracy_score(labels, preds)
    probs = torch.softmax(torch.tensor(logits), dim=1)[:,1].numpy()
    try:
        auc = roc_auc_score(labels, probs)
    except:
        auc = 0.0
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1, "auc": auc}

# 11. TRAINER
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))
        loss = loss_fct(outputs.logits, labels)
        return (loss, outputs) if return_outputs else loss

# 12. TRAINING ARGS (YÊU CẦU CỦA BẠN: GIỮ NGUYÊN ĐỂ SO SÁNH)
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=5,               # Giống DistilBERT
    per_device_train_batch_size=16,   # Giống DistilBERT (dù MiniLM có thể chịu được 32)
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    learning_rate=1e-2,               # LR 0.01
    warmup_ratio=0.1,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# 13. TRAIN
print("\n==============================")
print("🚀 TRAINING LIAR 2 - MiniLM + PROMPT TUNING")
print("==============================\n")
trainer.train()

# 14. EVALUATE
print("\n🎯 TEST RESULTS")
results = trainer.evaluate(tokenized["test"])
for k,v in results.items():
    print(f"{k}: {v}")

# 15. SAVE
final_path = os.path.join(OUTPUT_DIR, "final_liar2_pt")
trainer.save_model(final_path)
tokenizer.save_pretrained(final_path)
print(f"Saved to: {final_path}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.2 MB/s eta 0:00:00
Device: Tesla T4 | VRAM: 15.8 GB
Drive có thể đã được mount, tiếp tục...
Loading 'chengxuphd/liar2' dataset...


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/19.0M [00:00<?, ?B/s]

valid.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

...


Tham số huấn luyện (Prompt Tuning):
trainable params: 13,058 || all params: 22,727,044 || trainable%: 0.0575

🚀 TRAINING LIAR 2 - MiniLM + PROMPT TUNING



model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Auc
1,0.709300,0.726195,0.393557,0.154887,0.393557,0.222290,0.534684
2,0.706800,0.704372,0.393557,0.154887,0.393557,0.222290,0.557862
3,0.703000,0.694280,0.606443,0.367773,0.606443,0.457873,0.586992
4,0.694200,0.699365,0.483674,0.609765,0.483674,0.439925,0.588920
5,0.685200,0.680870,0.511972,0.586771,0.511972,0.500765,0.589368



🎯 TEST RESULTS


eval_loss: 0.6818442940711975
eval_accuracy: 0.5143728222996515
eval_precision: 0.5808246755053674
eval_recall: 0.5143728222996515
eval_f1: 0.5069026302641338
eval_auc: 0.5831990181553873
eval_runtime: 1.3599
eval_samples_per_second: 1688.345
eval_steps_per_second: 52.945
epoch: 5.0
Saved to: /content/drive/MyDrive/LIAR2_MiniLM_PromptTuning/final_liar2_pt
